In [14]:
import pandas as pd
import numpy as np
from math import *

def generar_puntos(centro, mat_cov, size):
  rng = np.random.default_rng()
  return rng.multivariate_normal(centro, mat_cov, size)

def generar_centro(d, val):
  return np.repeat(val, d)

def generar_clases(C, d, n):
  mat_cov = np.diag(np.full(d, C*sqrt(d)))
  size = n//2

  centro1 = generar_centro(d, 1)
  clase1 = generar_puntos(centro1, mat_cov, size)
  #print(clase1)

  centro2 = generar_centro(d, -1)
  clase2 = generar_puntos(centro2, mat_cov, size)
  #print(clase2)
  return (clase1, clase2)

C = 0.75
d = 2
n = 200

generar_clases(C, d, n)


(array([[ 2.57404599e-01,  3.05996520e-01],
        [-2.85147450e-01, -3.94086408e-01],
        [ 1.90672561e+00,  3.77737660e-01],
        [ 1.80482163e+00,  1.68875802e+00],
        [ 2.18739438e+00,  2.97030553e+00],
        [ 2.10290018e-02, -1.12986018e-01],
        [ 3.19005512e-02,  6.79575286e-03],
        [ 1.46054604e+00,  6.69386722e-01],
        [-3.18006952e-01, -1.26584185e-01],
        [ 1.02416687e+00, -1.40595495e-01],
        [ 3.46619223e+00, -2.05237203e-01],
        [ 3.96795384e-01,  2.04354895e-01],
        [-6.44871983e-02,  1.87607412e+00],
        [ 8.61803764e-01,  4.87014132e-01],
        [ 7.03208172e-01,  1.63849764e+00],
        [ 1.02335001e+00,  1.59009471e+00],
        [ 8.82801950e-01,  1.71357757e+00],
        [-5.64126496e-01,  7.06317853e-01],
        [ 1.52978787e+00,  4.66316193e+00],
        [ 1.06841833e-01,  1.44123889e+00],
        [ 3.03457891e+00,  9.40460481e-01],
        [ 2.94599724e+00,  7.71486523e-01],
        [ 3.38246933e-01, -5.825